# Libraries

In [1]:
import pandas as pd
from tqdm import tqdm
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from math import ceil
import torch
import numpy as np
from multiprocessing import Pool, cpu_count

# News dataset loading

In [4]:
tqdm.pandas()

In [5]:
path = "../data/news/nasdaq_external_data.csv"

news_df = pd.read_csv(path)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

# News dataset inpection

In [31]:
news_df.head()

,Unnamed: 0,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,0.0,2023-12-16 23:00:00 UTC,Interesting A Put And Call Options For August ...,A,https://www.nasdaq.com/articles/interesting-a-...,NaN,NaN,"Investors in Agilent Technologies, Inc. (Symbo...",Because the $125.00 strike represents an appro...,The current analytical data (including greeks ...,Below is a chart showing the trailing twelve m...,"At Stock Options Channel, our YieldBoost formu..."
1,1.0,2023-12-12 00:00:00 UTC,Wolfe Research Initiates Coverage of Agilent T...,A,https://www.nasdaq.com/articles/wolfe-research...,NaN,NaN,"Fintel reports that on December 13, 2023, Wolf...","Fintel reports that on December 13, 2023, Wolf...","T. Rowe Price Investment Management holds 10,1...",Agilent Technologies Declares $0.24 Dividend O...,The projected annual revenue for Agilent Techn...
2,2.0,2023-12-12 00:00:00 UTC,Agilent Technologies Reaches Analyst Target Price,A,https://www.nasdaq.com/articles/agilent-techno...,NaN,NaN,"In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...",When a stock reaches the target an analyst has...,When a stock reaches the target an analyst has...
3,3.0,2023-12-07 00:00:00 UTC,Agilent (A) Enhances BioTek Cytation C10 With ...,A,https://www.nasdaq.com/articles/agilent-a-enha...,NaN,NaN,Agilent Technologies A is enhancing its BioTek...,"Per a Grand View Research report, the global m...","Notably, Agilent enhanced the BioTek Cytation ...","Agilent Technologies, Inc. Price and Consensus...","Notably, Agilent enhanced the BioTek Cytation ..."
4,4.0,2023-12-07 00:00:00 UTC,"Pre-Market Most Active for Dec 7, 2023 : SQQQ,...",A,https://www.nasdaq.com/articles/pre-market-mos...,NaN,NaN,The NASDAQ 100 Pre-Market Indicator is up 70.2...,ProShares UltraPro Short QQQ (SQQQ) is -0.15 a...,"As reported by Zacks, the current mean recomme...","The total Pre-Market volume is currently 39,23...",The NASDAQ 100 Pre-Market Indicator is up 70.2...


In [32]:
print(len(news_df))
print(news_df.columns)

15549299
Index(['Unnamed: 0', 'Date', 'Article_title', 'Stock_symbol', 'Url',
       'Publisher', 'Author', 'Article', 'Lsa_summary', 'Luhn_summary',
       'Textrank_summary', 'Lexrank_summary'],
      dtype='object')


In [33]:
# Show the unique values of every column and their counts
for col in news_df.columns:
    print(f"Unique values in {col}: {news_df[col].nunique()}")
    print(news_df[col].unique())
    print()

Unique values in Unnamed: 0: 2491785
[0.000000e+00 1.000000e+00 2.000000e+00 ... 2.491783e+06 2.491784e+06
          nan]

Unique values in Date: 2796596
['2023-12-16 23:00:00 UTC' '2023-12-12 00:00:00 UTC'
 '2023-12-07 00:00:00 UTC' ... '2013-11-10 02:40:00 UTC'
 '2013-11-10 02:39:00 UTC' '2013-11-10 00:46:00 UTC']

Unique values in Article_title: 10452636
['Interesting A Put And Call Options For August 2024'
 'Wolfe Research Initiates Coverage of Agilent Technologies (A) with Outperform Recommendation'
 'Agilent Technologies Reaches Analyst Target Price' ...
 'Apple Said to Add Foxconn’s Chimei as Supplier of Parts for IPad'
 'CEZ, Bank Pekao Shares May Move: Central European Stock Preview'
 'Popolare di Milano May Join Intesa, Paschi in Selling Shares This Year']

Unique values in Stock_symbol: 8552
['A' 'AA' 'AAAU' ... 'XVZ' 'YGRO' 'YPRO']

Unique values in Url: 12745848
['https://www.nasdaq.com/articles/interesting-a-put-and-call-options-for-august-2024'
 'https://www.nasdaq.com/a

# News dataset refining (handling missing values)

In [3]:
# Drop columns Unnamed: 0 and Url
news_df = news_df.drop(columns=["Unnamed: 0", "Url"])

news_df.head()

,Date,Article_title,Stock_symbol,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,2023-12-16 23:00:00 UTC,Interesting A Put And Call Options For August ...,A,NaN,NaN,"Investors in Agilent Technologies, Inc. (Symbo...",Because the $125.00 strike represents an appro...,The current analytical data (including greeks ...,Below is a chart showing the trailing twelve m...,"At Stock Options Channel, our YieldBoost formu..."
1,2023-12-12 00:00:00 UTC,Wolfe Research Initiates Coverage of Agilent T...,A,NaN,NaN,"Fintel reports that on December 13, 2023, Wolf...","Fintel reports that on December 13, 2023, Wolf...","T. Rowe Price Investment Management holds 10,1...",Agilent Technologies Declares $0.24 Dividend O...,The projected annual revenue for Agilent Techn...
2,2023-12-12 00:00:00 UTC,Agilent Technologies Reaches Analyst Target Price,A,NaN,NaN,"In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...",When a stock reaches the target an analyst has...,When a stock reaches the target an analyst has...
3,2023-12-07 00:00:00 UTC,Agilent (A) Enhances BioTek Cytation C10 With ...,A,NaN,NaN,Agilent Technologies A is enhancing its BioTek...,"Per a Grand View Research report, the global m...","Notably, Agilent enhanced the BioTek Cytation ...","Agilent Technologies, Inc. Price and Consensus...","Notably, Agilent enhanced the BioTek Cytation ..."
4,2023-12-07 00:00:00 UTC,"Pre-Market Most Active for Dec 7, 2023 : SQQQ,...",A,NaN,NaN,The NASDAQ 100 Pre-Market Indicator is up 70.2...,ProShares UltraPro Short QQQ (SQQQ) is -0.15 a...,"As reported by Zacks, the current mean recomme...","The total Pre-Market volume is currently 39,23...",The NASDAQ 100 Pre-Market Indicator is up 70.2...


In [4]:
print(len(news_df))

# Drop rows with missing values in date and stock symbol columns
news_df = news_df.dropna(subset=["Date", "Stock_symbol"])

print(len(news_df))

15549299
5744672


In [5]:
print(len(news_df))

# Drop rows only if they have missing values in each of these columns: Article_title, Publisher, Author, Article, Lsa_summary, Luhn_summary, Textrank_summary, Lexrank_summary
news_df = news_df.dropna(subset=["Article_title", "Publisher", "Author", "Article", "Lsa_summary", "Luhn_summary", "Textrank_summary", "Lexrank_summary"], how="all")

print(len(news_df))

5744672
5744672


In [6]:
# Fill all missing values of the Article_title, Publisher, Author, Article, Lsa_summary, Luhn_summary, Textrank_summary, Lexrank_summary columns with "Unknown"
news_df[["Article_title", "Publisher", "Author", "Article", "Lsa_summary", "Luhn_summary", "Textrank_summary", "Lexrank_summary"]] = news_df[["Article_title", "Publisher", "Author", "Article", "Lsa_summary", "Luhn_summary", "Textrank_summary", "Lexrank_summary"]].fillna("Unknown")

In [7]:
# Check if there are any missing values in the DataFrame
missing_values = news_df.isnull().sum()
print("Missing values in each column:")
print(missing_values[missing_values > 0])

Missing values in each column:
Series([], dtype: int64)


In [8]:
news_df.head()

,Date,Article_title,Stock_symbol,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,2023-12-16 23:00:00 UTC,Interesting A Put And Call Options For August ...,A,Unknown,Unknown,"Investors in Agilent Technologies, Inc. (Symbo...",Because the $125.00 strike represents an appro...,The current analytical data (including greeks ...,Below is a chart showing the trailing twelve m...,"At Stock Options Channel, our YieldBoost formu..."
1,2023-12-12 00:00:00 UTC,Wolfe Research Initiates Coverage of Agilent T...,A,Unknown,Unknown,"Fintel reports that on December 13, 2023, Wolf...","Fintel reports that on December 13, 2023, Wolf...","T. Rowe Price Investment Management holds 10,1...",Agilent Technologies Declares $0.24 Dividend O...,The projected annual revenue for Agilent Techn...
2,2023-12-12 00:00:00 UTC,Agilent Technologies Reaches Analyst Target Price,A,Unknown,Unknown,"In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...",When a stock reaches the target an analyst has...,When a stock reaches the target an analyst has...
3,2023-12-07 00:00:00 UTC,Agilent (A) Enhances BioTek Cytation C10 With ...,A,Unknown,Unknown,Agilent Technologies A is enhancing its BioTek...,"Per a Grand View Research report, the global m...","Notably, Agilent enhanced the BioTek Cytation ...","Agilent Technologies, Inc. Price and Consensus...","Notably, Agilent enhanced the BioTek Cytation ..."
4,2023-12-07 00:00:00 UTC,"Pre-Market Most Active for Dec 7, 2023 : SQQQ,...",A,Unknown,Unknown,The NASDAQ 100 Pre-Market Indicator is up 70.2...,ProShares UltraPro Short QQQ (SQQQ) is -0.15 a...,"As reported by Zacks, the current mean recomme...","The total Pre-Market volume is currently 39,23...",The NASDAQ 100 Pre-Market Indicator is up 70.2...


In [9]:
# Cut the date column to only keep the date part keep only the first 10 characters
news_df["Date"] = news_df["Date"].str[:10]

In [10]:
news_df.head()

,Date,Article_title,Stock_symbol,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,2023-12-16,Interesting A Put And Call Options For August ...,A,Unknown,Unknown,"Investors in Agilent Technologies, Inc. (Symbo...",Because the $125.00 strike represents an appro...,The current analytical data (including greeks ...,Below is a chart showing the trailing twelve m...,"At Stock Options Channel, our YieldBoost formu..."
1,2023-12-12,Wolfe Research Initiates Coverage of Agilent T...,A,Unknown,Unknown,"Fintel reports that on December 13, 2023, Wolf...","Fintel reports that on December 13, 2023, Wolf...","T. Rowe Price Investment Management holds 10,1...",Agilent Technologies Declares $0.24 Dividend O...,The projected annual revenue for Agilent Techn...
2,2023-12-12,Agilent Technologies Reaches Analyst Target Price,A,Unknown,Unknown,"In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...",When a stock reaches the target an analyst has...,When a stock reaches the target an analyst has...
3,2023-12-07,Agilent (A) Enhances BioTek Cytation C10 With ...,A,Unknown,Unknown,Agilent Technologies A is enhancing its BioTek...,"Per a Grand View Research report, the global m...","Notably, Agilent enhanced the BioTek Cytation ...","Agilent Technologies, Inc. Price and Consensus...","Notably, Agilent enhanced the BioTek Cytation ..."
4,2023-12-07,"Pre-Market Most Active for Dec 7, 2023 : SQQQ,...",A,Unknown,Unknown,The NASDAQ 100 Pre-Market Indicator is up 70.2...,ProShares UltraPro Short QQQ (SQQQ) is -0.15 a...,"As reported by Zacks, the current mean recomme...","The total Pre-Market volume is currently 39,23...",The NASDAQ 100 Pre-Market Indicator is up 70.2...


In [ ]:
# Columns to include in Input_text
cols_to_use = [col for col in news_df.columns if col not in ["Date", "Stock_symbol"]]

def create_input_text_fast(row):
    return "\n".join(f"{col}: {getattr(row, col)}" for col in cols_to_use)

# Create Input_text using itertuples for better performance and memory usage
input_texts = []
for row in tqdm(news_df.itertuples(index=False), total=len(news_df)):
    input_texts.append(create_input_text_fast(row))

100%|██████████| 5744672/5744672 [04:41<00:00, 20392.74it/s] 


In [12]:
# Assign the new column
news_df["Input_text"] = input_texts

# Drop unused columns
news_df = news_df.drop(columns=[
    "Article_title", "Publisher", "Author", "Article", 
    "Lsa_summary", "Luhn_summary", "Textrank_summary", "Lexrank_summary"
])

news_df.head()

,Date,Stock_symbol,Input_text
0,2023-12-16,A,Article_title: Interesting A Put And Call Opti...
1,2023-12-12,A,Article_title: Wolfe Research Initiates Covera...
2,2023-12-12,A,Article_title: Agilent Technologies Reaches An...
3,2023-12-07,A,Article_title: Agilent (A) Enhances BioTek Cyt...
4,2023-12-07,A,Article_title: Pre-Market Most Active for Dec ...


In [13]:
# Save the DataFrame to a CSV file
news_df.to_csv("../data/news/processed.csv", index=False)

# Processed news loading

In [6]:
# Load list of stock symbols from the file
stock_symbols = [f[:-4] for f in os.listdir("../data/normalized/sp500/csv") if f.endswith(".csv")]
# Remove the .DS_Store file if it exists
if ".DS_Store" in stock_symbols:
    stock_symbols.remove(".DS_Store")

print(stock_symbols)
print(len(stock_symbols))

['A', 'AAL', 'AAP', 'AAPL', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'AMAT', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'AON', 'AOS', 'APA', 'APD', 'APH', 'ARE', 'ATVI', 'AVB', 'AVY', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF-A', 'BIIB', 'BIO', 'BK', 'BLK', 'BMRA', 'BMY', 'BR', 'BRK-A', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CCI', 'CDE', 'CDNS', 'CF', 'CHD', 'CHRW', 'CINF', 'CL', 'CLX', 'CME', 'CMG', 'CMI', 'CNC', 'CNP', 'COO', 'COP', 'COST', 'COWN', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CTAS', 'CTSH', 'CUK', 'D', 'DAL', 'DE', 'DFS', 'DGX', 'DHI', 'DIS', 'DLTR', 'DOV', 'DPZ', 'DRI', 'DTE', 'DVA', 'DXCM', 'EA', 'EBAY', 'ECL', 'ED', 'EFX', 'EIX', 'EL', 'EMN', 'EMR', 'ENS', 'EOG', 'EQIX', 'EQR', 'ES', 'ESS', 'EW', 'EXR', 'F', 'FAST', 'FCX', 'FDX', 'FE', 'FFIV', 'FIS', 'FISV', 'FITB', 'FLS', 'FMC', 'FRT', 'FTI', 'GD', 'GE', 'GGG', 'GILD', 'GIS', 'GOOG', 'GPC', 'GPN', 'GRMN', 'GWW', 'HAL', '

In [7]:
path = "../data/news/processed.csv"

news_df = pd.read_csv(path)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# Remove rows where Stock_symbol is not in the list of stock symbols
print(len(news_df))
news_df = news_df[news_df["Stock_symbol"].isin(stock_symbols)]
print(len(news_df))

5744672
1016283


In [ ]:
# Iterate over the stock symbols and create a separate CSV file for each
for stock_symbol in stock_symbols:
    print("Processing stock symbol:", stock_symbol)
    # Filter the DataFrame for the current stock symbol
    filtered_df = news_df[news_df["Stock_symbol"] == stock_symbol]
    
    # Save the filtered DataFrame to a CSV file
    filtered_df.to_csv(f"../data/news/single_stocks/{stock_symbol}.csv", index=False)

Processing stock symbol: A
Processing stock symbol: AAL
Processing stock symbol: AAP
Processing stock symbol: AAPL
Processing stock symbol: ABC
Processing stock symbol: ABMD
Processing stock symbol: ABT
Processing stock symbol: ACN
Processing stock symbol: ADI
Processing stock symbol: ADM
Processing stock symbol: ADP
Processing stock symbol: ADSK
Processing stock symbol: AEE
Processing stock symbol: AEP
Processing stock symbol: AIZ
Processing stock symbol: AJG
Processing stock symbol: AKAM
Processing stock symbol: ALB
Processing stock symbol: ALGN
Processing stock symbol: ALK
Processing stock symbol: AMAT
Processing stock symbol: AMD
Processing stock symbol: AME
Processing stock symbol: AMGN
Processing stock symbol: AMP
Processing stock symbol: AMT
Processing stock symbol: AMZN
Processing stock symbol: AON
Processing stock symbol: AOS
Processing stock symbol: APA
Processing stock symbol: APD
Processing stock symbol: APH
Processing stock symbol: ARE
Processing stock symbol: ATVI
Process

# FinBert model test

In [2]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
# Take first row to test
row = news_df.iloc[0]

# Create the input text by concatenating the columns with their names and values
input_text = ""
for col in news_df.columns:
    # Skip the columns that are not needed for the input text
    if not col in ["Date", "Stock_symbol"]:
        input_text += f"{col}: {row[col]}\n"
print(input_text)

NameError: name 'news_df' is not defined

In [ ]:
input_tokenized = tokenizer(input_text, return_tensors="pt")
print(input_tokenized)
print(len(input_tokenized["input_ids"][0]))

{'input_ids': tensor([[ 101, 3720, 1035,  ..., 5067, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}
1711


In [ ]:
sentiments = []

for i in range(ceil(len(input_tokenized["input_ids"][0]) / 512)):
    # Get the start and end indices for the current chunk
    start = i * 512
    end = min((i + 1) * 512, len(input_tokenized["input_ids"][0]))

    # Get the input IDs for the current chunk
    input_ids_chunk = input_tokenized["input_ids"][0][start:end].unsqueeze(0)

    # Get the attention mask for the current chunk
    attention_mask_chunk = input_tokenized["attention_mask"][0][start:end].unsqueeze(0)

    # Get the model output for the current chunk
    with torch.no_grad():
        output = model(input_ids_chunk, attention_mask=attention_mask_chunk)
        logits = output.logits
        # Convert the logits to probabilities
        probs = torch.nn.functional.softmax(logits, dim=-1)

    # Get the predicted sentiment for the current chunk
    sentiment = probs[0]
    # Append sentiment value and the length of the chunk that will be used as a weight for the average to the sentiments list
    sentiments.append((sentiment, end - start))

NameError: name 'input_tokenized' is not defined

In [ ]:
print(sentiments)

[(tensor([-0.4522, -1.1169,  2.0375]), 512), (tensor([-0.5175, -1.2336,  2.1715]), 512), (tensor([-0.3820, -1.3655,  1.9722]), 512), (tensor([-0.7449, -1.1740,  2.2532]), 175)]


In [50]:
# Average the sentiments using the weights
weighted_sentiment = sum([sentiment[0] * sentiment[1] for sentiment in sentiments]) / sum([sentiment[1] for sentiment in sentiments])
print(weighted_sentiment)

tensor([-0.4807, -1.2320,  2.0801])


In [13]:
test = [
    torch.tensor([0.1, 0.2, 0.3]),
    torch.tensor([0.4, 0.5, 0.6]),
    torch.tensor([0.7, 0.8, 0.9])
]

# Average the tensors
average_tensor = sum(test) / len(test)
print(average_tensor)

tensor([0.4000, 0.5000, 0.6000])


# Sentiment processing

In [20]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Move model to GPU if available
if torch.cuda.is_available():
    model = model.to("cuda")
    print("Model moved to GPU")

Model moved to GPU


In [21]:
def compute_sentiment(input_text):
    # Tokenize the input text without truncation (we'll handle long text manually)
    input_tokenized = tokenizer(input_text, return_tensors="pt", truncation=False)

    # Extract token IDs and attention mask from the tokenized input
    input_ids = input_tokenized["input_ids"][0]
    attention_mask = input_tokenized["attention_mask"][0]

    sentiments = []

    # Process the input in chunks of 512 tokens (maximum input size for BERT)
    for i in range(ceil(len(input_ids) / 512)):
        start, end = i * 512, min((i + 1) * 512, len(input_ids))

        # Extract the current chunk for input IDs and attention mask
        input_ids_chunk = input_ids[start:end].unsqueeze(0).to("cuda")         # Add batch dimension and move to GPU
        attention_mask_chunk = attention_mask[start:end].unsqueeze(0).to("cuda")

        # Disable gradient computation for inference (saves memory and speeds up)
        with torch.no_grad():
            output = model(input_ids_chunk, attention_mask=attention_mask_chunk)
            # Get the logits and softmax probabilities
            logits = output.logits
            # Apply softmax to get probabilities
            probs = torch.nn.functional.softmax(logits, dim=-1)

        # Move sentiment logits back to CPU and save them along with their weight (chunk length)
        sentiment = probs[0].cpu()
        sentiments.append((sentiment, end - start))  # Weight by chunk size

    # Compute a weighted average of the sentiments across all chunks
    weighted_sentiment = sum([s * w for s, w in sentiments]) / sum(w for _, w in sentiments)

    # Convert the tensor output to a Python list of floats
    return [float(x) for x in weighted_sentiment]


In [22]:
def compute_list_of_sentiments(texts):
    sentiments = []

    # Use tqdm to show progress
    for text in tqdm(texts):
        # Compute sentiment for each text
        sentiment = compute_sentiment(text)
        sentiments.append(sentiment)

    # Return the list of sentiments
    return sentiments

In [23]:
# Load list of stock symbols from the file
stock_symbols = [f[:-4] for f in os.listdir("../data/normalized/sp500/csv") if f.endswith(".csv")]
# Remove the .DS_Store file if it exists
if ".DS_Store" in stock_symbols:
    stock_symbols.remove(".DS_Store")

print(stock_symbols)

['A', 'AAL', 'AAP', 'AAPL', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'AMAT', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'AON', 'AOS', 'APA', 'APD', 'APH', 'ARE', 'ATVI', 'AVB', 'AVY', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF-A', 'BIIB', 'BIO', 'BK', 'BLK', 'BMRA', 'BMY', 'BR', 'BRK-A', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CCI', 'CDE', 'CDNS', 'CF', 'CHD', 'CHRW', 'CINF', 'CL', 'CLX', 'CME', 'CMG', 'CMI', 'CNC', 'CNP', 'COO', 'COP', 'COST', 'COWN', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CTAS', 'CTSH', 'CUK', 'D', 'DAL', 'DE', 'DFS', 'DGX', 'DHI', 'DIS', 'DLTR', 'DOV', 'DPZ', 'DRI', 'DTE', 'DVA', 'DXCM', 'EA', 'EBAY', 'ECL', 'ED', 'EFX', 'EIX', 'EL', 'EMN', 'EMR', 'ENS', 'EOG', 'EQIX', 'EQR', 'ES', 'ESS', 'EW', 'EXR', 'F', 'FAST', 'FCX', 'FDX', 'FE', 'FFIV', 'FIS', 'FISV', 'FITB', 'FLS', 'FMC', 'FRT', 'FTI', 'GD', 'GE', 'GGG', 'GILD', 'GIS', 'GOOG', 'GPC', 'GPN', 'GRMN', 'GWW', 'HAL', '

In [24]:
# TEST ONLY
# stock_symbols = stock_symbols[:5]
# print(stock_symbols)

In [25]:
# Iterate over the list of stock symbols
for stock in stock_symbols:
    print(f"\n🔍 Processing: {stock}")

    # Define file paths for the stock price CSV, the related news CSV, and the output file
    stock_file_path = f"../data/normalized_2/sp500/csv/{stock}.csv"
    news_file_path = f"../data/news/single_stocks/{stock}.csv"
    out_path = f"../data/normalized_2/with_sentiment/sp500/csv/{stock}.csv"

    # Load the stock price data
    stock_df = pd.read_csv(stock_file_path)

    # Check if the news file exists for the current stock
    if not os.path.exists(news_file_path):
        # If not, notify and fill sentiment columns with default value 1/3 (neutral sentiment)
        print("⚠️ News file missing — using default sentiments.")
        stock_df[["Sentiment_1", "Sentiment_2", "Sentiment_3"]] = 1/3
    else:
        # Load the news data
        news_df = pd.read_csv(news_file_path)

        # Group news by Date and Stock_symbol and concatenate all input texts for that group
        grouped = (
            news_df.groupby(["Date", "Stock_symbol"])["Input_text"]
            .apply(" ".join)  # Concatenate all Input_texts into one string per group
            .reset_index()    # Reset index to get a proper DataFrame
        )

        print("⚙️ Computing sentiment...")
        # Compute sentiment vectors in parallel for all grouped input texts
        sentiments = compute_list_of_sentiments(grouped["Input_text"].tolist())

        # Create a new DataFrame from the computed sentiment values
        sentiment_df = pd.DataFrame(sentiments, columns=["Sentiment_1", "Sentiment_2", "Sentiment_3"])

        # Combine the grouped (Date + Stock_symbol) with the new sentiment columns
        grouped = pd.concat([grouped.drop(columns=["Input_text"]), sentiment_df], axis=1)

        # Merge sentiment information into the stock dataframe on Date and Stock_symbol
        stock_df = pd.merge(stock_df, grouped, on=["Date", "Stock_symbol"], how="left")

        # Fill missing sentiment values with 0 (in case some dates had no news)
        stock_df[["Sentiment_1", "Sentiment_2", "Sentiment_3"]] = stock_df[["Sentiment_1", "Sentiment_2", "Sentiment_3"]].fillna(1/3)

    # Save the updated stock DataFrame with sentiment scores to the output path
    stock_df.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path}")


🔍 Processing: A
⚙️ Computing sentiment...


100%|██████████| 1143/1143 [00:20<00:00, 56.27it/s]


✅ Saved: ../data/normalized_2/with_sentiment/sp500/csv/A.csv

🔍 Processing: AAL
⚙️ Computing sentiment...


100%|██████████| 2284/2284 [04:21<00:00,  8.74it/s]


✅ Saved: ../data/normalized_2/with_sentiment/sp500/csv/AAL.csv

🔍 Processing: AAP
⚙️ Computing sentiment...


...

✅ Saved: ../data/normalized_2/with_sentiment/sp500/csv/YUM.csv

🔍 Processing: ZBH
⚙️ Computing sentiment...


100%|██████████| 543/543 [00:36<00:00, 14.86it/s]


✅ Saved: ../data/normalized_2/with_sentiment/sp500/csv/ZBH.csv

🔍 Processing: ZION
⚙️ Computing sentiment...


100%|██████████| 1352/1352 [00:49<00:00, 27.55it/s]


✅ Saved: ../data/normalized_2/with_sentiment/sp500/csv/ZION.csv
